# Декораторы

### Замыкания (closures)



Замыкание (*closures*) — функция с расширенной областью видимости, которая охватывает не глобальные
переменные, на которые есть ссылки в теле функции, хотя в теле функции они (переменные) не определенны.
Еще раз обратите внимание на то, что функция должна иметь доступ к неглобальным переменным. Т.е. к
переменным объявленным в другой (объемлющей) функции.

In [1]:
def add(q, w):
    return q + w

a = add
print(a(2, 3))

5


In [5]:
"""Область видимости параметра n принадлежит внешней функции, однако
вложенная функция может его использовать."""

def calculate_pow(n): # Объемлющая функция
    def calculate(number):
        print(locals())
        # Вложенная функция, которая использует переменную объемлющей
        return number ** n
    return calculate # Возврат вложенной функции

f = calculate_pow(3) # Вызов объемлющей функции




In [6]:
f

<function __main__.calculate_pow.<locals>.calculate(number)>

In [7]:
number_one = f(2) # Вызов вложенной функции
number_two = f(5)
print(number_one)
print(number_two)

{'number': 2, 'n': 3}
{'number': 5, 'n': 3}
8
125


In [9]:
func = calculate_pow(5)

In [10]:
func(3)

{'number': 3, 'n': 5}


243

In [11]:
f(4)

{'number': 4, 'n': 3}


64

внутренняя функция и является **замыканием**. В Python область видимости объемлющей функции
сохраняется для внутренней функции. Но не всех, а только для тех переменных которые используются во вложенной
функции. Такие переменные объемлющей функции называются свободными переменными.

In [12]:
# проблемы при построении функции замыкания
def fibonacci():
    first_number = 0
    second_number = 1
    def get_next():
        next_number = second_number + first_number # UnboundLocalError
        first_number = second_number
        second_number = next_number
        return next_number
    return get_next

In [13]:
f = fibonacci()

for i in range(10):
    print(f(), end = " ")

UnboundLocalError: local variable 'second_number' referenced before assignment

### Использование модификатора nonlocal

In [14]:
def fibonacci():
    first_number = 0
    second_number = 1
    def get_next():
        nonlocal second_number # указание, что переменные нелокальны
        nonlocal first_number
        next_number = second_number + first_number
        first_number = second_number
        second_number = next_number
        return next_number
    return get_next


In [15]:
f = fibonacci()

for i in range(10):
    print(f(), end = " ")


1 2 3 5 8 13 21 34 55 89 

## Декораторы функций

### Декоратор — это функция, которая позволяет обернуть другую функцию для расширения её функциональности без непосредственного изменения её кода.

**Декоратор** — вызываемый объект, который принимает другую функцию в качестве аргумента. Декоратор может производить
какие-то операции с функцией и возвращает либо ее саму, либо
другую, заменяющую ее функцию или вызываемый объект.
Декораторы функций связывают имя функции с другим вызываемым объектом на этапе определения функции, добавляя
дополнительный уровень логики, которая управляет функциями и методами или выполняет некоторые действия в случае их вызова.

Декораторы функций могут быть использованы для:

* Перехват вызова функций и выполняющие необходимые операции с функцией. Например, регистрация в прикладном компоненте, хронометраж.
* Декоратор может полностью заменить объект функции на другой. Или модифицировать объект функции.

Т.е. с помощью декоратора можно дополнять декорируемую функцию, можно использовать результаты ее работы, а можно полностью ее
заменить.

**Что может выступать в качестве декоратора в Python?**

Декоратор сам по себе является вызываемым объектом, который возвращает вызываемый объект. Т.е. в качестве
декоратора может использоваться любой объект, который реализует протокол вызова.
Если рассмотреть декораторы с технической точки зрения, то декораторы в Python - это синтаксическое упрощение
(синтаксический сахар) при описания объекта, который может управлять объектом функцией.
Однако, из-за своей наглядности, они применяются очень часто.

In [16]:
my_function = []

def add_function(func):
    """функция принимает на вход любой объект, добавляет
        его в список my_function и возвращает этот объект.
    """
    my_function.append(func)
    return func

@add_function # Применение созданной функции в качестве декоратора
def summ(x, y): # Декорируемая функция summ = add_function(sum)
    return x + y

@add_function
def mul(x, y): # Декорируемая функция
    return x * y

print(my_function)

[<function summ at 0x7fb100290d30>, <function mul at 0x7fb100290ee0>]


In [17]:
mul(4, 5)

20

In [18]:
def div(q, w):
    return q / w


In [19]:
div(5, 6)


0.8333333333333334

In [21]:
div = add_function(div)
# @add_function
# def div(q, w):
#     return q / w


In [22]:
print(my_function)

[<function summ at 0x7fb100290d30>, <function mul at 0x7fb100290ee0>, <function div at 0x7fb10032f160>]


декораторы - не более чем упрощение синтаксиса. Однако, из-за наглядности и удобства использования, их применяют
довольно часто.

**Внимание!** Декораторы выполняются на этапе создания функции. Декораторы не вызываются при вызове декорируемой
функции


### Передача параметров для декорируемой функции.
Пример декоратора, который использует результат вызова декорируемой функции

In [23]:
def to_str(func):
    def get_str(*args, **kwargs): # Функция, которая принимает аргументы для декор.функции
        return str(func(*args, **kwargs))
    return get_str # Теперь возвращается другая функция

@to_str
def suma(x, y):
    return x + y
print("Summa = " + suma(3, 4))

# suma = to_str(suma)
# suma = get_str(*args, **kwargs) -> str(f(*args, **kwargs))

Summa = 7


In [24]:
'h' + 7

TypeError: can only concatenate str (not "int") to str

в результате вызова функции декоратора возвращается уже не декорируемая функция, а иной вызываемый объект (функция get_str)

In [25]:
print(suma)

<function to_str.<locals>.get_str at 0x7fb1002a50d0>


In [26]:
@to_str
def div(q, w):
    return q / w

In [27]:
div

<function __main__.to_str.<locals>.get_str(*args, **kwargs)>

In [28]:
div(3, 5)

'0.6'

### Как это можно исправить?

In [29]:
def trace(func):
    """Декоратор trace выводит на экран сообщение с
        информацией о вызове декорируемой функции."""
    def inner(*args, **kwargs):
        """Inner doc"""
        print(f'name: {func.__name__}, args: {args}, kwargs: {kwargs}')
        return func(*args, **kwargs)
    return inner

In [30]:
@trace # identity = trace(identity)
def identity(x):
    """I do nothing useful."""
    return x

identity(50)



name: identity, args: (50,), kwargs: {}


50

In [31]:
identity

<function __main__.trace.<locals>.inner(*args, **kwargs)>

In [32]:
help(identity) # Help on function inner

Help on function inner in module __main__:

inner(*args, **kwargs)
    Inner doc



In [33]:
def identity(x):
    """I do nothing useful."""
    return x

identity.__name__, identity.__doc__

('identity', 'I do nothing useful.')

In [34]:
# @trace 
# def identity(x):
identity = trace(identity)


In [35]:
identity.__name__, identity.__doc__

('inner', 'Inner doc')

In [36]:
# У любой функции в Python есть атрибут __module__ ,
# содержащий имя модуля, в котором функция была определена.
identity.__module__

'__main__'

In [37]:
import math
print(math.cos.__module__)

math


In [38]:
# установим “правильные” значения в атрибуты декорируемой функции:
def trace(func):
    """Декоратор trace выводит на экран сообщение с
        информацией о вызове декорируемой функции."""
    def inner(*args, **kwargs):
        """Inner doc"""
        print(f'name: {func.__name__}, args: {args}, kwargs: {kwargs}')
        return func(*args, **kwargs)
    inner.__module__ = func.__module__
    inner.__name__ = func.__name__
    inner.__doc__ = func.__doc__
    return inner


@trace
def identity(x):
    """I do nothing useful."""
    return x

In [39]:
# Проверим
identity.__name__, identity.__doc__

('identity', 'I do nothing useful.')

In [42]:
dir(identity)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__wrapped__']

В модуле **functools** из стандартной библиотеки Python есть функция, реализующая логику копирования
внутренних атрибутов


In [40]:
import functools

def trace(func):
    """Декоратор trace выводит на экран сообщение с
        информацией о вызове декорируемой функции."""
    def inner(*args, **kwargs):
        """Inner doc"""
        print(f'name: {func.__name__}, args: {args}, kwargs: {kwargs}')
        return func(*args, **kwargs)
    functools.update_wrapper(inner, func)
    return inner

@trace
def identity(x):
    """I do nothing useful."""
    return x

identity.__name__, identity.__doc__

('identity', 'I do nothing useful.')

In [41]:
# То же самое можно сделать с помощью декоратора wraps

def trace(func):
    """Декоратор trace выводит на экран сообщение с
        информацией о вызове декорируемой функции."""
    @functools.wraps(func)
    def inner(*args, **kwargs):
        """Inner doc"""
        print(f'name: {func.__name__}, args: {args}, kwargs: {kwargs}')
        return func(*args, **kwargs)
    return inner

@trace
def identity(x):
    """I do nothing useful."""
    return x

print(identity.__name__, identity.__doc__)
print(identity(34))

identity I do nothing useful.
name: identity, args: (34,), kwargs: {}
34


### К одной функции можно применить множество декораторов

In [43]:
def bread(func):
    def wrapper():
        print()
        func()
        print("<\______/>")
    return wrapper

def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")
    return wrapper

def sandwich(food="--ветчина--"):
    print(food)

sandwich()

--ветчина--


In [44]:
sandwich = bread(ingredients(sandwich))
sandwich()


#помидоры#
--ветчина--
~салат~
<\______/>


In [45]:
@bread
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()


#помидоры#
--ветчина--
~салат~
<\______/>


In [46]:


@ingredients
@bread
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

#помидоры#

--ветчина--
<\______/>
~салат~


In [48]:
# важен порядок декорирования
@bread
@ingredients
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()



#помидоры#
#помидоры#
--ветчина--
~салат~
~салат~
<\______/>


### Аргументы декораторов
Обе разновидности декораторов (декораторы на основе функций и декораторы на
основе классов) могут принимать дополнительные аргументы.
Данный механизм реализован следующим образом: аргументы переданные
декоратору, на самом деле, передаются объекту, который вернет декоратор. А уже
возвращенный декоратор будет применен к декорируемому объекту.
Т.е. если использовать декоратор с параметрами, то, в качестве декоратора, стоит
использовать вызываемый объект, который вернет декоратор.

```
@my_decorator
def func(*args):
    ...
```
=>

`func = my_decorator(func)`

Для декоратора с параметрами, сохраняется логика, но добавляется
промежуточный уровень обработки
```
@my_decorator(x, y)
def func(*args):
    ...
```
=>
```
deco = my_decorator(x, y)
func = deco(func)
```

In [49]:
@bread('Hi')
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

TypeError: wrapper() takes 0 positional arguments but 1 was given

In [ ]:
def decorator_with_arguments(deco_arg1, deco_arg2):
    print("Я создаю декораторы! аргументы:", deco_arg1, deco_arg2)
    def my_decorator(func):
        print("Я и есть декоратор. Аргументы извне:", deco_arg1, deco_arg2)
        # Не перепутайте аргументы декораторов с аргументами функций!
        def wrapped(func_arg1, func_arg2):
            print ("Я - обёртка вокруг декорируемой функции.\n"
            "И я имею доступ ко всем аргументам\n"
            "\t- и декоратора: {0} {1}\n"
            "\t- и функции: {2} {3}\n"
            "Теперь я могу передать нужные аргументы дальше"
            .format(deco_arg1, deco_arg2, func_arg1, func_arg2))
            return func(func_arg1, func_arg2)
        return wrapped
    return my_decorator


In [ ]:
@decorator_with_arguments("Леонард", "Шелдон")
def decorated_function_with_arguments(_arg1, _arg2):
    print ("Я - декорируемая функция и я знаю только о своих аргументах: {0}"
    " {1}".format(_arg1, _arg2))

print('-------------------- start ---------------')
decorated_function_with_arguments("Раджеш", "Говард")

In [ ]:
# Декорирование добавит функцию в словарь workers.
# Ключом будет выступать строка (параметр декоратора),
# а значением - декорируемая функция.

workers = {}

def link(adress=None):
    def add_worker(func):
        workers[adress] = func
        def get_answer(*args, **kwargs):
            return str(func(*args, **kwargs))
        return get_answer
    return add_worker

In [ ]:
@link("\main")
def main_page():
    return "Hello word page"

@link("\main\goods")
def get_goods(list_goods):
    return list_goods

print(workers)

In [ ]:
@link()
def world():
    return "Hello world"
print(workers)

### Некоторые декораторы стандартной библиотеки
Рассмотрим некоторые декораторы в стандартной библиотеке:

**functools.lru_cache** — Встроенная реализация  мемоизации для пользовательских рекурсивных функций

**functools.singledispatch** — Реализация обобщенных (перегруженных) функций в Python

### Использование декоратора functools.lru_cache
Данный декоратор используется для реализации приема мемоизации («memoization»). Его смысл заключается в сохранении
параметра метода и его возвращаемого значения в быстром хранилище (словарь). Этот прием позволит значительно ускорить
вычисление некоторых рекурсивных функций. Так как, если в этом словаре уже будут параметры, с которыми вызывалась функция, то
она не будет вычисляться, а ответ возьмется из словаря.
У данного декоратора существует два параметра:

**maxsize** — сколько результатов вызова хранить. Для достижения максимальной производительности рекомендуется использовать два
в целой степени. По умолчанию maxsize=128

**typed** — по-разному хранить параметры разных типов. Т.е.
integer и float хранятся по разному. Например для чисел 3 и 3.0 данные будут храниться, как разные.

**Внимание!** Параметры функции должны быть хешируемого типа.

In [ ]:
import functools
import time

@functools.lru_cache()
def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

start = time.time()
res = fibonacci(250)
print(res)
print(time.time() - start)

In [ ]:
import time
def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

start = time.time()
res = fibonacci(36) # 12.167090892791748  75 будет кардинально дольше! 2 в степени n!
print(time.time() - start)

In [ ]:
def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    print(n)
    return fibonacci(n-1) + fibonacci(n-2)


res = fibonacci(10)

### Использование декоратора functoois.singledispatch
Данный декоратор используется для реализации приема
перегрузки методов. В Python не реализован механизм перегрузки
методов в зависимости от типа параметра. Однако необходимость в
подобном механизме возникает. В этом случае можно использовать
встроенный декоратор **functoois.singledispatch**. Функция, к которой
применен данный декоратор, становится обобщенной.
После этого вы можете зарегистрировать несколько других
функций для обработки аргумента своего типа. Для этого другие
функции следует пометить декоратором вида:

**decorate_function_name.register (date_type)**

**decorate_function_name** — имя обобщенной функции

**date_type** — тип данных

**Внимание!** Использовать декоратор можно только для работы с первым аргументом.

In [ ]:
import functools
import numbers

@functools.singledispatch
def add(a, b): # функция add будет обобщенной
    return a + b

@add.register(numbers.Integral) # если первый параметр add - целое число
def integer_add(a, b):
    print("Call for integer")
    return a + b

@add.register(numbers.Real) # если первый параметр add - вещественное число
def float_add(a, b):
    print("Call for float")
    return a + b

In [ ]:
# вызываются разные функции по одному и тому же имени
print(add(3, 5))
print(add(1.5, 0.5))

In [ ]:
# Использовать декоратор можно только для работы с первым аргументом.
print(add(3.0, 5))
print(add(15, 0.5))
